## 예스24 도서 데이터 크롤링

In [2]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import selenium
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException

In [3]:
# Selenium 옵션 설정
options = Options()
options.add_argument("--start-maximized")
options.add_experimental_option('detach', True)  # 스크립트가 종료된 후에도 브라우저가 닫히지 않도록 설정

# ChromeDriver 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# URL 포맷
url_template = 'https://www.yes24.com/Product/Category/MonthWeekBestSeller?categoryNumber={}&pageNumber=1&pageSize=30&type=month&saleYear=2024&monthNo={}&sex={}&age={}'

# 카테고리와 성별 설정
categories = {
    "001001001": "가정과 살림",
    "001001002": "자연과학",
    "001001008": "만화/라이트노벨",
    "001001010": "역사",
    "001001011": "건강 취미",
    "001001019": "인문",
    "001001025": "경제 경영",
    "001001026": "자기계발",
    "001001027": "유아",
    "001001046": "소설/시/희곡"
}

genders = ['F', 'M']
all_data = []
log_data = []

# Yes24 페이지 접속 및 데이터 수집
for category_code, category_name in categories.items():
        for month in range(1,7):
            for gender in genders:
                for age in range(10, 61, 10):  # 연령대 순회 설정
                    url = url_template.format(category_code, month, gender, age)
                    driver.get(url)                    
                    try:
                        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a.gd_name')))  # 책제목 기다리기
                        # 책 정보 추출
                        book_titles = [element.text.strip() for element in driver.find_elements(By.CSS_SELECTOR, 'a.gd_name')]  # 책제목
                        book_authors = [element.text.strip() for element in driver.find_elements(By.CSS_SELECTOR, 'span.authPub.info_auth')]  # 지은이
                        book_publisher = [element.text.strip() for element in driver.find_elements(By.CSS_SELECTOR, 'span.authPub.info_pub')]  # 출판사
                        book_prices = [element.text.strip() for element in driver.find_elements(By.CSS_SELECTOR, 'strong.txt_num')]  # 책가격
                        book_date = [element.text.strip() for element in driver.find_elements(By.CSS_SELECTOR, 'span.authPub.info_date')]  # 출판일
                        book_sales = [element.text.strip() for element in driver.find_elements(By.CSS_SELECTOR, 'span.saleNum')]  # 판매지수
                        book_ratings = [element.text.strip() for element in driver.find_elements(By.CSS_SELECTOR, 'span.rating_grade em.yes_b')]  # 평점
                        book_keywords = [element.text.strip().split('\n') for element in driver.find_elements(By.CSS_SELECTOR, 'div.info_row.info_tag')]  # 키워드
                        book_urls = [element.get_attribute('data-goods-no') for element in driver.find_elements(By.CSS_SELECTOR, 'li[data-goods-no]')]  # 상품 번호

                        time.sleep(1)  # 데이터 로딩을 위한 추가 대기

                        # 각 책의 상세 정보 수집 및 페이지 수 추출
                        for url in book_urls:
                            product_url = f"https://www.yes24.com/Product/Goods/{url}"
                            response = requests.get(product_url)
                            
                            if response.status_code == 200:
                                soup = BeautifulSoup(response.content, 'html.parser')
                                
                                # 페이지 수 추출
                                first_selector = '#infoset_specific > div.infoSetCont_wrap > div > table > tbody > tr:nth-child(2) > td'
                                second_selector = '#infoset_specific > div.infoSetCont_wrap > div > table > tbody > tr:nth-child(3) > td'
                                
                                # 첫 번째 선택자로 요소 선택
                                page_info_element = soup.select_one(first_selector)
                                
                                # 데이터가 없으면 두 번째 선택자로 요소 선택
                                if not page_info_element or "제본방식" in page_info_element.get_text(strip=True):
                                    page_info_element = soup.select_one(second_selector)
                                    
                                if page_info_element:
                                    page_info = page_info_element.get_text(strip=True)
                                    page_info = page_info.split('쪽')[0]

                            driver.get(product_url)  # 상세 페이지로 이동

                            # 데이터프레임에 추가할 데이터 정리
                            data = {
                                'Year': 2024,
                                'Month': month,
                                'Category': category_name,
                                'Title': book_titles[book_urls.index(url)],
                                'Author': book_authors[book_urls.index(url)],
                                'Publisher': book_publisher[book_urls.index(url)],
                                'Price': book_prices[book_urls.index(url)],
                                'Publish_Date': book_date[book_urls.index(url)],
                                'Sales_Index': book_sales[book_urls.index(url)],
                                'Rating': book_ratings[book_urls.index(url)],
                                'Pages': page_info,
                                'Keywords': book_keywords[book_urls.index(url)],
                                'Gender': '1' if gender == 'F' else '0',
                                'Age': age
                            }

                            # 데이터프레임 생성
                            df = pd.DataFrame([data])
                            df['Price'] = df['Price'].str.replace('[,원]', '', regex=True).astype(int)
                            df['Sales_Index'] = df['Sales_Index'].apply(lambda x: int(x.split(' ')[1].replace(',', '')))
                            df['month'] = 6

                            all_data.append(df)

                    except Exception as e:
                        log_data.append(f"Failed to fetch book information: {str(e)}")

driver.quit()

# 모든 데이터를 하나의 데이터프레임으로 합치기
all_data_df = pd.concat(all_data, ignore_index=True)

print(f"데이터 수집 및 저장이 완료되었습니다.")

데이터 수집 및 저장이 완료되었습니다.


In [29]:
all_data_df

,Category,Title,Author,Publisher,Price,Publish_Date,Sales_Index,Rating,Age_Scores,Pages,Keywords,Gender,Age,Comments,month
0,자기계발,당신이 꿈꾸지 못할 성공은 없다,빅 존슨 저/곽성혜 역,유노북스,15300,2024년 07월,300,9.6,{},264,"[#크레마클럽에있어요, #어른이된다는건, #심리학처방전]",1,10,원문주소 : https://sarak.yes24.com/review/15148762...,6
1,자기계발,생각이 너무 많은 어른들을 위한 심리학 (20만부 기념 리커버 에디션),김혜남 저,메이븐,16020,2023년 09월,234621,8.6,"{'10대': 0.0, '20대': 9.0, '30대': 9.0, '40대': 9....",320,"[#슬기로운언어생활, #어휘력, #크레마클럽에있어요]",1,10,원문주소 : https://sarak.yes24.com/review/18988713...,6
2,자기계발,어른의 어휘력,유선경 저,앤의서재,16920,2023년 05월,55410,8.9,"{'10대': 0.0, '20대': 9.0, '30대': 9.0, '40대': 9....",308,"[#크레마클럽에있어요, #인생지침서, #성공하고싶다면, #편집자의시대_편집자가만든책]",1,10,원문주소 : https://sarak.yes24.com/review/15148762...,6
3,자기계발,세상 끝의 카페,존 스트레레키 저/고상숙 역,클레이하우스,14850,2023년 02월,16461,9.0,"{'10대': 0.0, '20대': 10.0, '30대': 9.0, '40대': 9...",200,"[#올해의책, #성공신화, #성공하고싶다면, #삶이고민될때, #인생지침서]",1,10,원문주소 : https://sarak.yes24.com/review/18067440...,6
4,자기계발,세이노의 가르침,세이노(SayNo) 저,데이원,6480,2023년 03월,1235568,9.9,"{'10대': 10.0, '20대': 8.0, '30대': 9.0, '40대': 9...",736,[#크레마클럽에있어요],1,10,원문주소 : https://sarak.yes24.com/review/19669167...,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,자기계발,강인함의 힘,스티브 매그니스 저/이주만 역,상상스퀘어,19800,2024년 03월,82194,9.1,{},408,"[#유튜버가추천한책, #자기계발서, #나를바라보다, #성공하고싶다면]",0,30,원문주소 : https://sarak.yes24.com/review/19827074...,6
129,자기계발,마지막 몰입 (확장판) : 나를 넘어서는 힘,짐 퀵 저/김미정 역,비즈니스북스,16200,2024년 03월,10866,9.5,"{'10대': 0.0, '20대': 9.0, '30대': 9.0, '40대': 9....",496,"[#성공하고싶다면, #자기계발서, #부자비법, #삶을바꿀이야기, #부자만들기]",0,30,원문주소 : https://sarak.yes24.com/review/14019562...,6
130,자기계발,부의 인사이트,함서경 저,생각지도,18000,2024년 03월,21111,9.9,"{'10대': 0.0, '20대': 0.0, '30대': 10.0, '40대': 9...",312,"[#동기부여, #성공하고싶다면, #성공신화, #성공학]",0,30,원문주소 : https://sarak.yes24.com/review/19549686...,6
131,자기계발,똑똑한 사람은 어떻게 생각하고 질문하는가,이시한 저,북플레저,17820,2024년 01월,29961,9.3,"{'10대': 8.0, '20대': 10.0, '30대': 9.0, '40대': 9...",328,"[#성공학, #성공의공식, #성공하는사업의비결, #자수성가비법, #크레마클럽에있어요]",0,30,원문주소 : https://sarak.yes24.com/review/19870988...,6


In [1]:
# CSV 파일로 저장
all_data_df.to_csv('yes24_bestseller_data.csv', index=False, encoding='utf-8-sig')
# log.to_csv('log_datra.csv', index=False, encoding='utf-8-sig')

NameError: name 'all_data_df' is not defined

In [67]:
all_data_df.head()

,Category,Title,Author,Publisher,Price,Publish_Date,Sales_Index,Rating,Age_Scores,Pages,Keywords,Year,Month,Gender,Age,Comments
0,자기계발,세이노의 가르침,세이노(SayNo) 저,데이원,6480,2023년 03월,1235568,9.0,"{'10대': 10.0, '20대': 8.0, '30대': 9.0, '40대': 9...",736,"[#올해의책, #성공신화, #성공하고싶다면, #삶이고민될때, #인생지침서]",2023,6,1,10,원문주소 : https://sarak.yes24.com/review/19669167...
1,자기계발,데일 카네기 인간관계론 (무삭제 완역본),데일 카네기 저/임상훈 역,현대지성,10350,2019년 10월,242535,9.6,"{'10대': 10.0, '20대': 9.0, '30대': 9.0, '40대': 9...",352,"[#서탐대실, #크레마클럽에있어요, #북클러버의선택, #요즘책방:책읽어드립니다, #...",2023,6,1,10,원문주소 : https://sarak.yes24.com/review/17376573...
2,자기계발,원씽 THE ONE THING,"게리 켈러, 제이 파파산 저/구세희 역",비즈니스북스,15120,2013년 08월,221520,9.4,"{'10대': 9.0, '20대': 9.0, '30대': 9.0, '40대': 9....",양장 도서 제본방식 안내,"[#스페셜에디션, #성공하고싶다면, #일하는방식, #생활습관, #국립중앙도서관사서추천]",2023,6,1,10,원문주소 : https://sarak.yes24.com/review/10986385...
3,자기계발,역행자 확장판,자청 저,웅진지식하우스,17550,2023년 05월,329775,8.5,"{'10대': 10.0, '20대': 9.0, '30대': 9.0, '40대': 9...",400,"[#성공학, #성공의공식, #성공하는사업의비결, #자수성가비법, #창업할때읽으면좋은책]",2023,6,1,10,원문주소 : https://sarak.yes24.com/review/18194577...
4,자기계발,데일 카네기 자기관리론,데일 카네기 저/임상훈 역,현대지성,10350,2021년 01월,152799,9.7,"{'10대': 9.0, '20대': 9.0, '30대': 9.0, '40대': 9....",424,"[#페이커가읽은책, #서탐대실, #크레마클럽에있어요, #자기관리, #건강한자기관리]",2023,6,1,10,원문주소 : https://sarak.yes24.com/review/15692993...


In [64]:
all_data_df.to_csv('Yes24_books_dataset.csv', index=False, encoding='utf-8-sig')
# log_df.to_csv('Yes24_crawling_history.csv', index=False)